In [ ]:
# Imports

# Matemaricos, mapiulacao de dados e funcoes
import numpy as np
import pandas as pd
import time
# Desligando alertas de chained_assignment
pd.options.mode.chained_assignment = None  # default='warn'

# Graficos
#import matplotlib
#import matplotlib.pyplot as plt
#import seaborn as sns

# Chamadas e tratamentos de dados e arquivos
import re
import requests
from pathlib import Path

# Desativacao de warning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# GetDate
from datetime import date

In [ ]:
# Preenchimento do campo Data_Projeto
count = 1
failed_req= 0
failed = False
max_failed_req = 5
notfound_count = 0

data_projeto = ''
notfound_count_dt = 0

tramitacao_encerrada = ''
notfound_count_tram = 0

aprovado = ''
notfound_count_aprv = 0

while count > 0:
    for i in listagem.index:
        if listagem['Data_Projeto'][i] == '' or listagem['Tramitacao_Encerrada'][i] == '' or listagem['Aprovado'][i] == '': 
            data_projeto = ''
            tramitacao_encerrada = ''
            aprovado = ''

            failed = False
            # Variaveis de requisicao
            id_proj = listagem['Id_Projeto'][i]
            
            try:
                # Requisicao para o portal de propostas da Camara
                query = {"num": id_proj}
                resposta = requests.get('https://www.camarajf.mg.gov.br/sal/proposicao.php?', params=query)
                
                #Identificacao da data de registro do projeto
                try:
                    # Limpeza para pegar o id do projeto dentro da prefeitura
                    retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                    data_projeto = re.search('PER&IACUTE;ODO:(.+?)A', retorno).group(1)
                except:
                    data_projeto = 'Data nao encontrada'
                    notfound_count_dt += 1
                
                
                #Identificacao do encerramento da tramitacao do projeto
                try:
                    # Limpeza para pegar o id do projeto dentro da prefeitura
                    retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                    #tramitacao_encerrada = re.search('FIMTRAMITA&CCEDIL;&ATILDE;O:(.+?)\t\t\t\t\t\t', retorno).group(1)
                    if(re.search('FIMTRAMITA&CCEDIL;&ATILDE;O:SIM', retorno)):
                        tramitacao_encerrada = 'SIM'
                    else:
                        tramitacao_encerrada = 'NAO'
                except:
                    tramitacao_encerrada = 'Informacao nao encontrada'
                    notfound_count_tram+=1
                    
                    
                #Identificacao da aprovacao do projeto
                if listagem['Tramitacao_Encerrada'][i] == 'NAO': 
                    aprovado = 'TRAMITACAO'

                elif listagem['Codigo_Tipo'][i] not in ('REQ', 'MOC', 'REP', 'PEDIF'):
                    if listagem['Situacao'][i] != 'Arquivada' or 'Vetado' not in listagem['Situacao'][i]: # era 'Vetado Totalmente'
                        aprovado = 'SIM'
                    else:
                        aprovado = 'NAO'                

                elif listagem['Codigo_Tipo'][i] in ('REQ', 'MOC', 'REP', 'PEDIF'): 
                    try:
                        # Limpeza para pegar o id do projeto dentro da prefeitura
                        retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))

                        if(re.search('APROVAD', retorno)):
                            aprovado = 'SIM'
                        else:
                            aprovado = 'NAO'
                except:
                    aprovado = 'Informacao nao encontrada'
                    notfound_count_aprv += 1
                
                
                
            except:
                failed = True
                time.sleep(180)

            listagem['Data_Projeto'][i] = data_projeto
            listagem['Tramitacao_Encerrada'][i] = tramitacao_encerrada
            listagem['Aprovado'][i] = aprovado
            
            resposta.close()
            time.sleep(6)
            
        if failed:
            failed_req += 1
        else:
            filed_req = 0
                
#       Se houve 5 falhas consecutivas
        if failed_req > max_failed_req:
            break
            
    if failed_req > max_failed_req:
        break               
            
    count = 0          

    for i in listagem.index:
        if listagem['Data_Projeto'][i] == '' and listagem['Id_Projeto'][i] != '':
            count +=1

if failed_req > max_failed_req:
    print(f'O endpoint nao esta respondendo, verifique a conexao. Fim do processo. Nao encontrados = {notfound_count}')
else:
    print(f'Fim do processo. Nao encontrados = {notfound_count}')